# Imports 

In [1]:
import pandas as pd
import numpy as np
import os

## repeated printouts
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## Read in data and make sure relevant columns are string/character 

- San Diego data: `naics_code` and `account_key`
- NAICS details data: `naics` 

Run code below; if pulling from github, pathname should be fine; if working elsewhere may need to edit path name at read in 

In [3]:
# uncomment if u need to find your path: os.getcwd()
SD = sd_df = pd.read_csv("../../public_data/sd_df.csv")
NA = naics_df = pd.read_csv("../../public_data/naics_df.csv")

In [4]:
cols_sd_use = ["naics_code", "account_key"]
cols_naics_use = ["naics"]

sd_df[cols_sd_use] = sd_df[cols_sd_use].astype(str)
naics_df[cols_naics_use] = naics_df[cols_naics_use].astype(str)

sd_df.dtypes
naics_df.dtypes
sd_df.head()
naics_df.head()

account_key          object
dba_name             object
council_district     object
naics_code           object
naics_description    object
naics_nchar           int64
dtype: object

naics                object
naics_description    object
dtype: object

,account_key,dba_name,council_district,naics_code,naics_description,naics_nchar
0,1974000448,ERNST & YOUNG LLP,cd_1,541211,OFFICES OF CERTIFIED PUBLIC ACCOUNTANTS,6
1,1974011093,HECHT SOLBERG ROBINSON GOLDBERG & BAGLEY LLP,cd_3,5411,LEGAL SERVICES,4
2,1978039819,RSM US LLP,cd_1,541211,OFFICES OF CERTIFIED PUBLIC ACCOUNTANTS,6
3,1978042092,THORSNES BARTOLOTTA MCGUIRE LLP,cd_3,5411,LEGAL SERVICES,4
4,1979046817,KORENIC & WOJDOWSKI LLP,cd_7,5412,ACCOUNTING/TAX PREP/BOOKKEEP/PAYROLL SERVICES,4


,naics,naics_description
0,111140,Wheat Farming
1,111160,Rice Farming
2,111150,Corn Farming
3,111110,Soybean Farming
4,111120,Oilseed (except Soybean) Farming


## "Inner join"- retain only San Diego businesses with details on their NAICS code

- Use the `naics_code` column in the San Diego business data as the join key
- Use the `naics` column in the NAICS code details data as the join key

- Do an inner join of the San Diego data onto the NAICS code details using these join keys
- After the inner join, print some examples of San Diego businesses lost in the merge
- Use value_counts() on the `naics_nchar` column in the San Diego data to see why they might have gotten lost


In [5]:
M = SD.merge(NA, left_on="naics_code", right_on="naics", how="inner")

In [6]:
lost_sd = SD[~SD.naics_code.isin(M.naics_code)]

In [7]:
len(SD), len(M)

(353, 52)

In [8]:
SD[~SD.naics_code.isin(M.naics_code)].shape
SD[SD.naics_code.isin(M.naics_code)].shape

(318, 6)

(35, 6)

In [9]:
SD[~SD.naics_code.isin(M.naics_code)]

,account_key,dba_name,council_district,naics_code,naics_description,naics_nchar
1,1974011093,HECHT SOLBERG ROBINSON GOLDBERG & BAGLEY LLP,cd_3,5411,LEGAL SERVICES,4
3,1978042092,THORSNES BARTOLOTTA MCGUIRE LLP,cd_3,5411,LEGAL SERVICES,4
4,1979046817,KORENIC & WOJDOWSKI LLP,cd_7,5412,ACCOUNTING/TAX PREP/BOOKKEEP/PAYROLL SERVICES,4
5,1979053082,GRIMM VRANJES GREER STEPHAN & BRIDGMAN LLP,cd_3,5411,LEGAL SERVICES,4
6,1981000840,BENINK & SLAVENS LLP,cd_7,5411,LEGAL SERVICES,4
...,...,...,...,...,...,...
347,2006015404,9625 TOWN CENTRE PARTNERS LP,cd_3,541,"PROFESSIONAL, SCIENTIFIC & TECHNICAL SERVICES",3
348,2007015303,WFP-GATEWAY LP,cd_3,541,"PROFESSIONAL, SCIENTIFIC & TECHNICAL SERVICES",3
349,2007015336,WFP-MIRAMAR LP,cd_3,541,"PROFESSIONAL, SCIENTIFIC & TECHNICAL SERVICES",3
350,2010010820,ROSSLYN LOFTS HOUSING PARTNERS LP THE,cd_6,541,"PROFESSIONAL, SCIENTIFIC & TECHNICAL SERVICES",3


In [10]:
SD.naics_nchar.value_counts()

naics_nchar
5    150
4     89
6     49
3     42
2     23
Name: count, dtype: int64

In [11]:
df = df1.merge(df2, left_on= ["left_key_1", "left_key_2"], 
               right_on=["right_key_1", "right_key_2"], 
               how= "inner")

df = TIKTOK.merge(INSTAGRAM, left_on= ["user1", "user2"], 
               right_on=["user1", "user2"], 
               how= "inner")

NameError: name 'df1' is not defined

## "Left join"- retain all sd businesses even if naics code isn't in `naics_df`

- Using the same join keys as above, and treating the San Diego businesses as the left hand side data, left join the naics code details onto the San Diego businesses
- Use the `indicator` argument within merge to create an indicator, `naics_merge_status`, to help with later merge diagnostics and examine sample of ones that didn't merge
- Use the `suffixes` argument within merge to add `_sd` as the left suffix, `_census` as the right suffix
- Use `naics_merge_status` in the merged result to look at a sample of San Diego businesses that weren't matched with `naics_df`

In [12]:
M = SD.merge(NA, left_on="naics_code", 
             right_on="naics", how="left",
            indicator="naics_merge_status")
M.naics_merge_status.value_counts()

naics_merge_status
left_only     318
both           52
right_only      0
Name: count, dtype: int64

In [16]:
M = SD.merge(NA, left_on="naics_code", 
             right_on="naics", how="left",
            indicator="naics_merge_status",
            suffixes=("_sd","_census"))

In [15]:
M.naics_merge_status.value_counts(normalize = True)

naics_merge_status
left_only     0.859459
both          0.140541
right_only    0.000000
Name: proportion, dtype: float64

## Use group by and agg to see if there are differences in merge rates by area

- Using the left-joined dataframe created in previous step, create a boolean indicator, `is_lost`, that equals `True` if the merge indicator is equal to "left_only" (and is otherwise false)
- Find the proportion of businesses by council district that were lost in the left join (SD businesses that failed to match to `naics_df`). To do this, group by `council_district` and use the shortcut of taking the mean of the `is_lost` indicator to find the proportions lost by `council_district`. 

In [17]:
M["is_lost"] = M.naics_merge_status == "left_only"
M.groupby("council_district")["is_lost"].mean()

council_district
cd_1    0.880597
cd_2    0.804348
cd_3    0.812500
cd_4    1.000000
cd_5    0.916667
cd_6    0.859649
cd_7    0.918919
cd_8    0.933333
cd_9    1.000000
Name: is_lost, dtype: float64

## Optional challenge exercise: add lagging 0's and see if merge rate improves

You noticed earlier that a big reason for non-matches is that the San Diego tax certificate NAICS codes were often less than six digits long, while the Census ones were always 6 digits.

You wonder if this is an issue where 0's in some of the SD's data naics codes got cut off (eg 540000 became 54), and if so whether adding these lagging zeros would improve the merge rate in a left join.

- Pad the `naics_code` column in `sd_df` with 0's to get that column up to 6-digits, using one of two approaches: 
  1. `str.pad` in pandas (https://pandas.pydata.org/docs/reference/api/pandas.Series.str.pad.html)
  2. for more of a challenge, write your own function! It should check the # of digits in the naics code string and pad it with 0's at the end up to 6 digits. To execute your function, use row-wise apply: `df.apply(lambda row: funcname(row.column), axis=1)`.
- Perform the same left join as above and see how the match rate changes
- Create an indicator variable--`is_new_match`---for new matches under the padded NAICS code; compare the `naics_description` column from San Diego versus Census in the new dataset for a sample of these new matches and comment on whether the padding seems to be correct

In [ ]:
sd_df["naics_code"] = sd_df["naics_code"].str.pad(width=6, side="right", fillchar="0")
new_left = sd_df.merge(naics_df, left_on="naics_code", right_on="naics", how="left", 
                   indicator="naics_merge_status", suffixes=["_sd", "_census"])
new_left["is_lost"] = [True if x == "left_only" else False for x in new_left["naics_merge_status"]]
new_left.groupby("council_district")["is_lost"].agg("mean")

def conditions(df):
    if (df["is_lost_new"] == False and df["is_lost_old"] == True):
        return True
    else:
        return False

matches = left.merge(new_left, on="account_key", how="inner", suffixes=["_old", "_new"])
matches["is_new_match"] = matches.apply(conditions, axis=1)
matches[matches["is_new_match"] == True][["account_key", "dba_name_old", "naics_description_sd_new", "naics_description_census_new"]]

In [26]:
sd_df["naics_code_new"] = sd_df["naics_code"].str.pad(width=6, side="right", fillchar="0")
new_left = sd_df.merge(naics_df, left_on="naics_code_new", right_on="naics", how="left", 
                   indicator="naics_merge_status", suffixes=["_sd", "_census"])
new_left["is_lost"] = [True if x == "left_only" else False for x in new_left["naics_merge_status"]]
new_left.groupby("council_district")["is_lost"].agg("mean")

council_district
cd_1    0.546667
cd_2    0.555556
cd_3    0.348148
cd_4    1.000000
cd_5    0.483871
cd_6    0.445946
cd_7    0.560976
cd_8    0.333333
cd_9    1.000000
Name: is_lost, dtype: float64

In [22]:
M[ M.council_district == "cd_9" ]

,account_key,dba_name,council_district,naics_code,naics_description_sd,naics_nchar,naics,naics_description_census,naics_merge_status,is_lost
105,2007003007,LOS PRIMOS MEXICAN FOOD,cd_9,72221,LIMITED-SERVICE EATING PLACES,5,NaN,NaN,left_only,True
116,2007015787,MOREHEAD PARTNERS,cd_9,5313,ACTIVITIES RELATED TO REAL ESTATE,4,NaN,NaN,left_only,True
149,2009000691,LOS PRIMOS MEXICAN FOOD,cd_9,722211,LIMITED-SERVICE RESTAURANTS,6,NaN,NaN,left_only,True
226,2017016749,OROFACIAL THERAPEUTICS LP,cd_9,42145,MEDICAL/DENTAL/HOSPITAL EQUIP & SUPP WHSLE,5,NaN,NaN,left_only,True
266,2019015990,CLK 1 JUICE LP,cd_9,7222,LIMITED-SERVICE EATING PLACES,4,NaN,NaN,left_only,True
307,2020007615,HENCHMEN SVPPLI CO,cd_9,42,WHOLESALE TRADE,2,NaN,NaN,left_only,True
331,2020011851,FM WORKSPACE SOLUTIONS LLC,cd_9,44,RETAIL TRADE,2,NaN,NaN,left_only,True
350,2021006164,JAMBA JUICE #3353,cd_9,722213,SNACK & NONALCOHOLIC BEVERAGE BARS,6,NaN,NaN,left_only,True


In [58]:
def conditions(df):
    if (df["is_lost_new"] == False and df["is_lost_old"] == True):
        return True
    else:
        return False

matches = M.merge(new_left, on="account_key", how="inner", suffixes=["_old", "_new"])
matches["is_new_match"] = matches.apply(conditions, axis=1)
matches[matches["is_new_match"] == True][["account_key", "dba_name_old", "naics_description_sd_new", "naics_description_census_new"]]

/var/folders/0w/2wvbn9xx39scmwd7157fzlw80000gn/T/ipykernel_4064/3953952778.py:7: FutureWarning: Passing 'suffixes' which cause duplicate columns {'naics_code_new'} in the result is deprecated and will raise a MergeError in a future version.
  matches = M.merge(new_left, on="account_key", how="inner", suffixes=["_old", "_new"])


KeyError: 'is_lost_new'

In [25]:
SD["naics_padded"]

0      541211
1      541100
2      541211
3      541100
4      541200
        ...  
348    541000
349    541000
350    541000
351    811198
352    531000
Name: naics_padded, Length: 353, dtype: object

In [24]:
SD["naics_padded"] = SD.naics_code.str.pad(width=6,
                                          side="right",
                                          fillchar="0")

In [61]:
pad_zeros = lambda x: "NAICS: "+ x + "0"*(6-len(x))

In [62]:
SD["naics_padded_2"] = SD.naics_code.apply(pad_zeros)